In [1]:
import pandas as pd 
import numpy as np
from keras.models import model_from_json
import datetime
import io
from urllib.request import urlopen
from itertools import islice
from stldecompose import decompose
from matplotlib import pyplot as plt

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def rolling_window(a, window, step_size):
    shape = a.shape[:-1] + (a.shape[-1] - window +2  - step_size, window)
    strides = a.strides + (a.strides[-1] * step_size,)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [3]:
in_win = 15
out_win = 12

In [4]:
# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

### Selecting 1 years time frame till last day

In [5]:
e_date = datetime.date.today()-datetime.timedelta(1)
s_date = datetime.date.today()-datetime.timedelta(367)
#sp_date = datetime.date.today()-datetime.timedelta(7)
end_date = str(e_date.year) + str(e_date.strftime('%m')) + str(e_date.strftime('%d'))
start_date = str(s_date.year) + str(s_date.strftime('%m')) + str(s_date.strftime('%d'))
#split_date = str(sp_date.year) + '-' + str(sp_date.strftime('%m')) + '-' + str(sp_date.strftime('%d'))


#### Select currency

In [6]:
currency = "bitcoin"

In [7]:
data_bt = pd.read_html("https://coinmarketcap.com/currencies/" + currency +"/historical-data/?start=" + start_date + "&end=" + end_date)[0]
data_bt['Date'] = pd.to_datetime(data_bt['Date'])
data_bt = data_bt.sort_values(by=['Date']).reset_index(drop=True)
data_bt.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,2017-09-27,3892.94,4210.05,3884.82,4200.67,1686880000,64579231744
1,2017-09-28,4197.13,4279.31,4109.70,4174.73,1712320000,69633212416
2,2017-09-29,4171.62,4214.63,4039.29,4163.07,1367049984,69219221504
3,2017-09-30,4166.11,4358.43,4160.86,4338.71,1207449984,69136629760
4,2017-10-01,4341.05,4403.74,4269.81,4403.74,1208210048,72047304704


In [8]:
date_col = pd.to_datetime(data_bt["Date"])
data_open = data_bt["Open*"]
data_high = data_bt["High"]
data_low = data_bt["Low"]
data_close = data_bt["Close**"]
data_vol = data_bt["Volume"]
data_mark = data_bt["Market Cap"]

### Log Transforming, Decomposing , deseasonalizing and normalizing the input

In [9]:
log_open = np.log(data_open)
log_high = np.log(data_high)
log_low = np.log(data_low)
log_close = np.log(data_close)
log_vol = np.log(data_vol)
log_mark = np.log(data_mark)

In [10]:
log_open = pd.DataFrame(log_open)
log_open = log_open.set_index(date_col)

log_high = pd.DataFrame(log_high)
log_high = log_high.set_index(date_col)

log_low = pd.DataFrame(log_low)
log_low = log_low.set_index(date_col)

log_close = pd.DataFrame(log_close)
log_close = log_close.set_index(date_col)

log_vol = pd.DataFrame(log_vol)
log_vol = log_vol.set_index(date_col)

log_mark = pd.DataFrame(log_mark)
log_mark = log_mark.set_index(date_col)

In [11]:
stl_open = decompose(log_open)
stl_high = decompose(log_high)
stl_low = decompose(log_low)
stl_close = decompose(log_close)
stl_vol = decompose(log_vol)
stl_mark = decompose(log_mark)

In [12]:
deseason_open  = (stl_open.resid + stl_open.trend).iloc[:,0]
deseason_high  = (stl_high.resid + stl_high.trend).iloc[:,0]
deseason_low   = (stl_low.resid + stl_low.trend).iloc[:,0]
deseason_close = (stl_close.resid + stl_close.trend).iloc[:,0]
deseason_vol   = (stl_vol.resid + stl_vol.trend).iloc[:,0]
deseason_mark  = (stl_mark.resid + stl_mark.trend).iloc[:,0]

In [13]:
date_col_win_in = date_col[(len(date_col)-in_win):len(date_col)]
date_col_win_in_exp = np.expand_dims(date_col_win_in, axis=2)
date_col_win_in_exp = np.expand_dims(date_col_win_in_exp, axis=0)

data_open_win_in = deseason_open[(len(deseason_open)-in_win):len(deseason_open)]
data_open_win_in = pd.DataFrame(data_open_win_in) 
norm_open_win_in = data_open_win_in.subtract(data_open_win_in.iloc[in_win-1], axis='columns')
print(norm_open_win_in.shape)
norm_open_win_in_exp = np.expand_dims(norm_open_win_in.values, axis=0)
print(norm_open_win_in_exp.shape)

data_high_win_in = deseason_high[(len(deseason_high)-in_win):len(deseason_high)]
data_high_win_in = pd.DataFrame(data_high_win_in) 
norm_high_win_in = data_high_win_in.subtract(data_high_win_in.iloc[in_win-1], axis='columns')
norm_high_win_in_exp = np.expand_dims(norm_high_win_in.values, axis=0)

data_low_win_in = deseason_low[(len(deseason_low)-in_win):len(deseason_low)]
data_low_win_in = pd.DataFrame(data_low_win_in) 
norm_low_win_in = data_low_win_in.subtract(data_low_win_in.iloc[in_win-1], axis='columns')
norm_low_win_in_exp = np.expand_dims(norm_low_win_in.values, axis=0)

data_close_win_in = deseason_close[(len(deseason_close)-in_win):len(deseason_close)]
data_close_win_in = pd.DataFrame(data_close_win_in) 
norm_close_win_in = data_close_win_in.subtract(data_close_win_in.iloc[in_win-1], axis='columns')
norm_close_win_in_exp = np.expand_dims(norm_close_win_in.values, axis=0)

data_vol_win_in = deseason_vol[(len(deseason_vol)-in_win):len(deseason_vol)]
data_vol_win_in = pd.DataFrame(data_vol_win_in) 
norm_vol_win_in = data_vol_win_in.subtract(data_vol_win_in.iloc[in_win-1], axis='columns')
norm_vol_win_in_exp = np.expand_dims(norm_vol_win_in.values, axis=0)

data_mark_win_in = deseason_mark[(len(deseason_mark)-in_win):len(deseason_mark)]
data_mark_win_in = pd.DataFrame(data_mark_win_in) 
norm_mark_win_in = data_mark_win_in.subtract(data_mark_win_in.iloc[in_win-1], axis='columns')
norm_mark_win_in_exp = np.expand_dims(norm_mark_win_in.values, axis=0)


(15, 1)
(1, 15, 1)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  from ipykernel import kernelapp as app


### Stacking all inputs together

In [14]:
x_all = np.dstack((norm_open_win_in_exp, norm_high_win_in_exp, norm_low_win_in_exp
                   ,norm_close_win_in_exp, norm_vol_win_in_exp, norm_mark_win_in_exp))
x_all.shape

(1, 15, 6)

### Carrying out prediction

In [15]:
y_pred = model.predict(x_all)

### Taking seasonality of past 12 days

In [16]:
season_high = stl_high.seasonal
season_high = season_high[(len(season_high)-out_win):len(season_high)]

season_low = stl_low.seasonal
season_low = season_low[(len(season_low)-out_win):len(season_low)]

season_close = stl_close.seasonal
season_close = season_close[(len(season_close)-out_win):len(season_close)]


### Renormalizing, Reseasonalizing and antilog of prediction

In [17]:
high_pred  = pd.DataFrame(y_pred[:,:,0])
low_pred   = pd.DataFrame(y_pred[:,:,1])
close_pred = pd.DataFrame(y_pred[:,:,2])

In [18]:
high_denorm_pred  = high_pred.add(data_open_win_in.iloc[in_win-1].reset_index(drop=True), axis='index')
low_denorm_pred   = low_pred.add(data_open_win_in.iloc[in_win-1].reset_index(drop=True), axis='index')
close_denorm_pred = close_pred.add(data_open_win_in.iloc[in_win-1].reset_index(drop=True), axis='index')

In [19]:
deseason_high_pred  = high_denorm_pred.add(season_high.iloc[:,0].reset_index(drop=True), axis='columns')
deseason_low_pred   = low_denorm_pred.add(season_low.iloc[:,0].reset_index(drop=True), axis='columns')
deseason_close_pred = close_denorm_pred.add(season_close.iloc[:,0].reset_index(drop=True), axis='columns')

In [20]:
pred_high  = np.exp(deseason_high_pred)
pred_low   = np.exp(deseason_low_pred)
pred_close = np.exp(deseason_close_pred)

In [32]:
%matplotlib qt
plt.figure()
plt.plot(pred_high.T)
plt.figure()
plt.plot(pred_low.T)
plt.figure()
plt.plot(pred_close.T)
plt.show()